In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Layer
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
data_dir = '/content/IMAGES'

validation_split = 0.3
seed = 125
image_size = [224, 224]
batch_size = 1
epochs = 30
lr = 0.0002

In [ ]:
ds_train_ = image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="training",
    seed=seed,
    labels='inferred',
    label_mode='categorical',
    image_size=image_size,
    interpolation='nearest',
    batch_size=batch_size,
    shuffle=True,
)

ds_valid_ = image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="validation",
    seed=seed,
    labels='inferred',
    label_mode='categorical',
    image_size=image_size,
    interpolation='nearest',
    batch_size=batch_size,
    shuffle=True,
)

ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

Found 75 files belonging to 9 classes.
Using 53 files for training.
Found 75 files belonging to 9 classes.
Using 22 files for validation.


In [ ]:
class MyConvClass(tf.keras.Model):
    def __init__(self):
        super(MyConvClass, self).__init__()
        self.conv = models.Sequential([
                        tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
                        tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
                        tf.keras.layers.MaxPooling2D((64, 64), strides=(64, 64)),
                        tf.keras.layers.Flatten(),
                        ])
        self.classifier = models.Sequential([
                            layers.Dense(units = 512, activation='tanh'),
                            layers.Dropout(rate = 0.5),
                            layers.Dense(units = 256, activation='tanh'),
                            layers.Dense(units = 9, activation='softmax') # Totally two classes, so units = 2
                            ])

    def call(self, inputs, training=False):
          inputs = self.conv(inputs)
          outputs = self.classifier(inputs)
          return outputs

In [ ]:
model = MyConvClass()
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=lr),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(ds_train, validation_data=ds_valid, epochs=epochs)

Epoch 1/30
53/53 [==============================] - 4s 39ms/step - loss: 2.3800 - accuracy: 0.1132 - val_loss: 2.1233 - val_accuracy: 0.1364
Epoch 2/30
53/53 [==============================] - 2s 32ms/step - loss: 2.1471 - accuracy: 0.2830 - val_loss: 1.9533 - val_accuracy: 0.3182
Epoch 3/30
53/53 [==============================] - 2s 35ms/step - loss: 1.8679 - accuracy: 0.3019 - val_loss: 1.7394 - val_accuracy: 0.3182
Epoch 4/30
53/53 [==============================] - 2s 33ms/step - loss: 1.6538 - accuracy: 0.4340 - val_loss: 1.4870 - val_accuracy: 0.4545
Epoch 5/30
53/53 [==============================] - 2s 33ms/step - loss: 1.5564 - accuracy: 0.4528 - val_loss: 1.3632 - val_accuracy: 0.5000
Epoch 6/30
53/53 [==============================] - 2s 33ms/step - loss: 1.2966 - accuracy: 0.6792 - val_loss: 1.1804 - val_accuracy: 0.5000
Epoch 7/30
53/53 [==============================] - 2s 33ms/step - loss: 1.1709 - accuracy: 0.6415 - val_loss: 1.0521 - val_accuracy: 0.5455
Epoch 8/30
53

In [ ]:
model.save('/content/my_model.keras')